### Restart and Run All Cells

In [1]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"
dts_path = "/Users/User/Downloads/Datasets/"

# Directory to be monitored
directory = '/Users/User/Downloads'
# Dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png'],
    'PDFs': ['pdf'],
    'Datasets': ['csv', 'xlsx', 'xls', 'json'],
    'Videos': ['mp4','m4a'],
    'Zips': ['zip'],
    'EXEs': ['exe', 'msi'],
    'Pythons': ['ipynb'],
}
pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format
year = 2023
month = 10
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 10, 12), datetime.date(2023, 10, 11))

In [2]:
num_business_days = BDay(2)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2023-10-12
yesterday: 2023-10-10


## Must rerun whenever there is change in orders

In [3]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(48, 8)

In [4]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
input_file

'../data/Price-Today.csv'

In [5]:
prices = pd.read_csv(input_file)
prices.shape

(190, 10)

In [6]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(48, 17)

In [7]:
df_merge['spd'] = 0

In [8]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(48, 12)

In [9]:
file_name = "Price-for-Order.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,ASK,-32,RD40pct,SET,1500,18.0,21.2,-0.1,-0.469484,1,2023-02-28
B,BBL,-29,XXX,SET50,400,155.0,169.5,1.5,0.892857,0,2023-09-06
B,BEM,-8,XXX,SET50,2500,7.8,8.2,0.05,0.613497,1,2022-12-22
B,BH,-14,XXX,SET50,100,250.0,264.0,1.0,0.380228,1,2022-12-22
B,CK,-26,XXX,SET100,1000,17.8,20.4,0.1,0.492611,0,2022-12-22
B,CPN,-10,52WL,SET50,300,61.0,63.5,-0.5,-0.78125,1,2022-12-22
B,DELTA,-18,XXX,SET50,200,80.0,84.5,-1.5,-1.744186,1,2022-12-22
B,DIF,-8,RD40pct,SET,3000,8.65,9.05,-0.05,-0.549451,1,2023-08-15
B,GVREIT,-12,RD35pct,SET,6000,5.15,5.75,-0.05,-0.862069,1,2023-08-28
B,HANA,-39,XXX,SET100,400,55.0,64.75,-0.5,-0.766284,1,2022-12-22
B,ICHI,-24,XXX,SET,1500,14.3,16.7,0.1,0.60241,0,2022-12-22
B,III,-4,XXX,SET,2000,10.8,11.2,0.0,0.0,1,2022-12-22
B,IVL,-28,RD45pct,SET50,800,22.5,25.75,0.0,0.0,1,2023-08-28
B,JASIF,-6,RD35pct,SET,4000,6.5,6.8,0.0,0.0,1,2023-08-21
B,JMT,-5,RD35pct,SET50,800,36.75,38.0,-2.75,-6.748466,1,2023-0

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name

df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
27,B,SINGER,0,RD55pct,SET100,3000,11.00,11.00,-0.60,-5.17,1,2023-08-03,33000.00
33,B,WHAIR,-3,DOS,SET,4000,6.00,6.15,-0.05,-0.81,1,2023-08-18,24000.00


In [15]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
41,S,JMART,31,5pct,SET50,2000,24.10,21.00,-0.90,-4.11,2,2023-08-24,48200.00


In [16]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,ASK,-32,RD40pct,SET,1500,18.00,21.20,-0.10,-0.47,1,2023-02-28,27000.00
2,B,BEM,-8,XXX,SET50,2500,7.80,8.20,0.05,0.61,1,2022-12-22,19500.00
3,B,BH,-14,XXX,SET50,100,250.00,264.00,1.00,0.38,1,2022-12-22,25000.00
5,B,CPN,-10,52WL,SET50,300,61.00,63.50,-0.50,-0.78,1,2022-12-22,18300.00
6,B,DELTA,-18,XXX,SET50,200,80.00,84.50,-1.50,-1.74,1,2022-12-22,16000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,B,TFFIF,-5,RD10pct,SET,2500,6.60,6.85,0.00,0.00,1,2023-09-01,16500.00
31,B,TMT,-6,RD40pct,SET,3000,6.30,6.60,0.00,0.00,1,2023-08-24,18900.00
32,B,TTB,-6,XXX,SET50,20000,1.65,1.71,0.01,0.59,1,2022-12-22,33000.00
33,B,WHAIR,-3,DOS,SET,4000,6.00,6.15,-0.05,-0.81,1,2023-08-18,24000.00


In [17]:
df.query('name == "IVL"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
12,B,IVL,-28,RD45pct,SET50,800,22.50,25.75,0.00,0.00,1,2023-08-28,18000.00


In [18]:
pd.options.display.max_rows = 35
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
1,B,BBL,-29,XXX,SET50,400,155.00,169.50,1.50,0.89,0,2023-09-06,62000.00
4,B,CK,-26,XXX,SET100,1000,17.80,20.40,0.10,0.49,0,2022-12-22,17800.00
10,B,ICHI,-24,XXX,SET,1500,14.30,16.70,0.10,0.60,0,2022-12-22,21450.00
15,B,KTB,-16,XXX,SET50,1000,18.00,19.60,0.00,0.00,0,2023-04-20,18000.00
18,B,ORI,-14,RD15pct,SET100,3000,9.40,10.30,0.10,0.98,0,2023-08-28,28200.00
20,B,PTG,-10,RD25pct,SET100,3600,8.40,8.90,-0.10,-1.11,0,2022-12-22,30240.00
24,B,SAPPE,-19,XXX,SET,300,80.00,84.75,0.75,0.89,0,2022-12-22,24000.00
29,B,SYNEX,-12,RD65pct,SET,2000,10.00,11.20,-0.20,-1.75,0,2023-08-25,20000.00
34,B,WHART,-16,RD30pct,SET,5000,8.80,9.60,-0.10,-1.03,0,2023-08-21,44000.00
35,S,AH,18,5pct,SET,1200,38.75,34.25,0.50,1.48,0,2023-08-29,46500.00


In [19]:
df.query('spd <= -40 and trade == "B"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [20]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [21]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

file_name = "orders-log.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [22]:
df['percent'] = df['%change']

In [23]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
14,B,JMT,-5,RD35pct,SET50,800,36.75,38.00,-2.75,-6.75,1,2023-08-24,29400.00,-6.75
27,B,SINGER,0,RD55pct,SET100,3000,11.00,11.00,-0.60,-5.17,1,2023-08-03,33000.00,-5.17
41,S,JMART,31,5pct,SET50,2000,24.10,21.00,-0.90,-4.11,2,2023-08-24,48200.00,-4.11


In [24]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
17,B,NER,-11,RD40pct,SET,3000,4.44,4.66,0.10,2.19,1,2023-08-25,13320.00,2.19


In [25]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
27,B,SINGER,0,RD55pct,SET100,3000,11.00,11.00,-0.60,-5.17,1,2023-08-03,33000.00,-5.17
33,B,WHAIR,-3,DOS,SET,4000,6.00,6.15,-0.05,-0.81,1,2023-08-18,24000.00,-0.81
26,B,SENA,-4,RD45pct,SET,7000,2.56,2.64,-0.02,-0.75,1,2023-08-28,17920.00,-0.75
11,B,III,-4,XXX,SET,2000,10.80,11.20,0.00,0.00,1,2022-12-22,21600.00,0.00
14,B,JMT,-5,RD35pct,SET50,800,36.75,38.00,-2.75,-6.75,1,2023-08-24,29400.00,-6.75
30,B,TFFIF,-5,RD10pct,SET,2500,6.60,6.85,0.00,0.00,1,2023-09-01,16500.00,0.00
31,B,TMT,-6,RD40pct,SET,3000,6.30,6.60,0.00,0.00,1,2023-08-24,18900.00,0.00
13,B,JASIF,-6,RD35pct,SET,4000,6.50,6.80,0.00,0.00,1,2023-08-21,26000.00,0.00
32,B,TTB,-6,XXX,SET50,20000,1.65,1.71,0.01,0.59,1,2022-12-22,33000.00,0.59
19,B,PSH,-7,XXX,SET,2000,12.20,12.90,0.00,0.00,1,2023-03-10,24400.00,0.00


### Sell recently purchased stocks if profitable

In [26]:
pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(113, 4)

In [27]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [28]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(113, 8)

In [29]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending='False')

,name,date_x,qty,cost,price,difa,%change,%profit


### Profit per Day

In [30]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2023 AND MONTH(sells.date) = 10
ORDER BY sells.date DESC, name


In [31]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days


In [32]:
print(f"Monthly Profit Amount: {sells_df.gross.sum():,.2f}")

Monthly Profit Amount: 0.00


In [33]:
file_name = "monthly-sales.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [34]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day


In [35]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: nan%
